In [79]:
import requests
import pandas as pd
import locale
import json
import config
import datetime
import time
import mysql.connector
import pickle
import 

api_key = 'faf8be11d952e504b113126fb62259c6'

In [80]:
cnx = mysql.connector.connect(
            host = config.host,
            user = config.user,
            passwd = config.password)

cursor = cnx.cursor()

In [3]:
TABLES = {}

TABLES['movies'] = ("""CREATE TABLE john (
                                 imdbid VARCHAR(25),
                                 movieTitle VARCHAR(255) NOT NULL,
                                 released DATE NOT NULL,
                                 genreId VARCHAR(14),
                                 genreName VARCHAR(33),
                                 rating REAL,
                                 PRIMARY KEY (movieTitle, released)
                                 ) ENGINE=InnoDB""")


In [82]:
def add_tables_to_db(TABLES, cursor, db_name):
    from mysql.connector import errorcode
    for table_name in TABLES:
        table_description = TABLES[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(f"""USE {db_name}""")
            cursor.execute(table_description)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

In [83]:
add_tables_to_db(TABLES, cursor, 'Moovies')

Creating table movies: already exists.
Creating table movie_genres: already exists.
Creating table genreid: already exists.


In [4]:
TABLES['movie_genres'] = ("""CREATE TABLE movie_genres (
                                 mg_id INTEGER AUTO_INCREMENT,
                                 imdbid VARCHAR(25),
                                 genreId VARCHAR(14),
                                 PRIMARY KEY (mg_id)
                                 ) ENGINE=InnoDB""")

In [5]:
TABLES['genreid'] = ("""CREATE TABLE genreid (
                                 genreId VARCHAR(14),
                                 genreName VARCHAR(33),
                                 PRIMARY KEY (genreId)
                                 ) ENGINE=InnoDB""")

In [7]:
# iterating through a pages
popular_movies = []
for i in range(1,992):
    try:
        pnum = i
    #     url = f'https://api.themoviedb.org/3/discover/movie?api_key=faf8be11d952e504b113126fb62259c6&certification_country=US&certification.lte=G&sort_by=popularity.desc&page={pnum}'
        url = f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={pnum}'
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers)
        data = response.json()['results']
        popular_movies.extend(data)
#         if i % 40 == 0:
        time.sleep(1)
    except KeyError:
        pass
        if e.errno != errno.ECONNRESET:
            raise

In [8]:
# iterating through the movies list to parse the movie_id
def movie_list(popular_movies):
    popular_list = []
    for popular in popular_movies:
        popular_list.append(popular['id'])
    return popular_list


movie_id = movie_list(popular_movies)   
    
#used the movie id to iterate through a full detailed list of movies    

In [9]:
def movie_details(movie_id):
    detailed_list = []
#     detailed_lists = {}
    for movie in movie_id:
        url2 = f'https://api.themoviedb.org/3/movie/{movie}?api_key=faf8be11d952e504b113126fb62259c6&language=en-US'
        response2 = requests.get(url2, headers = headers)
        data2 = response2.json()
        detailed_list.append(data2)
    # we want to ADD all the detail lists togther
    return detailed_list


In [43]:
filename = 'movie_details'

In [44]:
outfile = open(filename, 'wb')

In [45]:
pickle.dump(full_list, outfile)

In [46]:
outfile.close()

In [47]:
infile = open(filename, 'rb')
p = pickle.load(infile)
infile.close()

In [36]:
# 100 at a time

# 19820 results

full_list = []
a = 0
b = 100
while b <= 19820:
    new = movie_details(movie_id[a:b])
    full_list.extend(new)
    a += 100
    b += 100
    time.sleep(1)
else:
    full_list.extend(movie_id[19800:19820])
    
# a = 19800, b = 19900

In [73]:
for movie in full_list:
    for i in movie['genres']:
        print(movie['imdb_id'],i['id'])

tt6105098 12
tt6105098 16
tt6105098 10751
tt6105098 18
tt6105098 28
tt4154796 12
tt4154796 878
tt4154796 28
tt9501310 16
tt9501310 28
tt9501310 18
tt9501310 9648
tt9501310 35
tt6320628 28
tt6320628 12
tt6320628 878
tt6806448 28
tt0437086 28
tt0437086 878
tt0437086 53
tt0437086 12
tt4154756 12
tt4154756 28
tt4154756 878
tt2139881 35
tt2139881 10749
tt0317219 16
tt0317219 12
tt0317219 35
tt0317219 10751
tt7131622 18
tt7131622 35
tt7131622 28
tt7131622 80
tt7131622 37
tt3741700 878
tt3741700 28
tt1979376 12
tt1979376 16
tt1979376 35
tt1979376 10751
tt4154664 28
tt4154664 12
tt4154664 878
tt0448115 28
tt0448115 35
tt0448115 14
tt6146586 80
tt6146586 28
tt6146586 53
tt5884052 28
tt5884052 12
tt5884052 14
None 28
None 12
None 878
tt2283336 28
tt2283336 35
tt2283336 878
tt2283336 12
tt8022928 12
tt8022928 10751
tt8022928 10402
tt10656752 18
tt10656752 10770
tt2274648 28
tt2274648 12
tt2274648 14
tt2274648 27
tt2274648 878
tt6139732 12
tt6139732 14
tt6139732 10749
tt6139732 35
tt6139732 10751


KeyError: 'genres'

In [86]:
def add_genre(full_list):
    for movie in full_list:
        try:
            for i in movie['genres']:
#                 then place the id and name into a genre table, 
                cursor.execute('''
                      INSERT INTO movie_genres (imdbid, genreId)
                       VALUES(%s, %s)''', (movie['imdb_id'], i['id']))
#                 print(i['id'],i['name'])
        except KeyError:
            pass
        except TypeError:
            print('issue with this: ', movie)
    cnx.commit()

In [ ]:
def add_genreid(m):
    genres = []
    for movie in range(len(m)):
        try:
            for i in m[movie]['genres']:
#                 then place the id and name into a genre table, 
                cursor.execute('''
                      INSERT IGNORE INTO genreid (genreId, genreName)
                       VALUES( %s, %s)''', (i['id'], i['name']))
#                 print(i['id'],i['name'])
                genres.append({m[movie]['title'] : (i['id'], i['name'])})
        except KeyError:
            pass
        
    cnx.commit()

In [ ]:
add_genreid(m)

In [ ]:
def add_movie(m):
    movie = []
    for ids in m:
        try:
            cursor.execute('''
              INSERT IGNORE INTO john (imdbid, movieTitle, released, rating )
              VALUES( %s, %s, %s, %s)''', (ids['imdb_id'],ids['title'], ids['release_date'],ids['vote_average']))
#             print(ids['id'],ids['title'],ids['imdb_id'], ids['release_date'], ids['vote_average'], int(ids['vote_count']))
            movie.append({ids['id'] : ids['title']})
        except KeyError:
            pass

    cnx.commit()

In [ ]:
add_movie(m)

In [ ]:
cnx.commit()